# Exercise 3 - Geometric transformations

In [ ]:
import copy
import numpy as np 
from PIL import Image

## Objective

In this exercise, you will implement the following geometric transformations
from scratch: horizontal flipping and resizing in `augmentations.py`. You can also 
implement random cropping as an additional but not required exercise. Your 
implementations should not only affect the images but also the associated bounding boxes. 

![](data/example.png)

In [ ]:
def calculate_iou(gt_bbox, pred_bbox):
    """
    calculate iou 
    args:
    - gt_bbox [array]: 1x4 single gt bbox
    - pred_bbox [array]: 1x4 single pred bbox
    returns:
    - iou [float]: iou between 2 bboxes
    - [xmin, ymin, xmax, ymax]
    """
    xmin = np.max([gt_bbox[0], pred_bbox[0]])
    ymin = np.max([gt_bbox[1], pred_bbox[1]])
    xmax = np.min([gt_bbox[2], pred_bbox[2]])
    ymax = np.min([gt_bbox[3], pred_bbox[3]])
    
    intersection = max(0, xmax - xmin) * max(0, ymax - ymin)
    gt_area = (gt_bbox[2] - gt_bbox[0]) * (gt_bbox[3] - gt_bbox[1])
    pred_area = (pred_bbox[2] - pred_bbox[0]) * (pred_bbox[3] - pred_bbox[1])
    
    union = gt_area + pred_area - intersection
    return intersection / union, [xmin, ymin, xmax, ymax]

## Details

The `hflip` function takes the image and bounding boxes as input and performs a 
horizontal flip. For example, an object initially on the left of the image will 
end up on the right.

In [ ]:
def hflip(img, bboxes):
    """
    horizontal flip of an image and annotations
    args:
    - img [PIL.Image]: original image
    - bboxes [list[list]]: list of bounding boxes
    return:
    - flipped_img [PIL.Image]: horizontally flipped image
    - flipped_bboxes [list[list]]: horizontally flipped bboxes
    """
    # IMPLEMENT THIS FUNCTION
    return flipped_img, flipped_bboxes

The `resize` function takes the image, bounding boxes and target size as input. 
It scales up or down images and bounding boxes.


In [ ]:
def resize(img, boxes, size):
    """
    resized image and annotations
    args:
    - img [PIL.Image]: original image
    - boxes [list[list]]: list of bounding boxes
    - size [array]: 1x2 array [width, height]
    returns:
    - resized_img [PIL.Image]: resized image
    - resized_boxes [list[list]]: resized bboxes
    """
    # IMPLEMENT THIS FUNCTION
    return resized_image, resized_boxes

The `random_crop` function takes a few additional inputs. It also needs the classes, 
the crop size and the minimum area. Let's explain these parameters:
* `crop_size` is the size of the crop. It should be smaller than the dimensions of the input image.
* `min_area` is the minimum area of a bounding boxes to be considered as an object after cropping.

In [ ]:
def random_crop(img, boxes, crop_size, min_area=100):
    """
    random cropping of an image and annotations
    args:
    - img [PIL.Image]: original image
    - boxes [list[list]]: list of bounding boxes
    - crop_size [array]: 1x2 array [width, height]
    - min_area [int]: min area of a bbox to be kept in the crop
    returns:
    - cropped_img [PIL.Image]: resized image
    - cropped_boxes [list[list]]: resized bboxes
    """
    # IMPLEMENT THIS FUNCTION
    return cropped_image, cropped_boxes

Because we are cropping randomly, we may only keep a tiny portion of an object, in which
case the annotations will not be useful anymore. For example, in the image below, we may not want to keep the annotation of the cat because most of the animal's body is not visible.

![](data/cat_cropped.png)

**Note:** You'll need to use the "Desktop" button to view the visualizations of each augmentation.

In [ ]:
# fix seed to check results
    
# open annotations
    
# filter annotations and open image
    
# check horizontal flip, resize and random crop

# use check_results defined in utils.py for this

## Tips

The `hflip` transform does not affect the x coordinates of the bounding boxes.

You will use the same ratio in `resize` for the image and the bounding boxes. 

To find which bounding box belongs to the cropped area, you can use the `calculate_iou`
function.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Rectangle
from PIL import Image


def plot_histogram(img):
    """ plot channel-wise pixel value distribution """
    histogram = img.histogram()

    R = histogram[0:256]
    G = histogram[256:512]
    B = histogram[512:768]

    plt.plot(range(256), R, color='r')
    plt.fill_between(range(256), R, color='r', alpha=0.5)
    plt.plot(range(256), G, color='g')
    plt.fill_between(range(256), G, color='g', alpha=0.5)
    plt.plot(range(256), B, color='b')
    plt.fill_between(range(256), B, color='b', alpha=0.5)
    plt.show()


def display_results(img, bboxes, aug_img, aug_bboxes):
    f, ax = plt.subplots(1, 2, figsize=(10, 10))
    ax[0].imshow(img)
    for bb in bboxes:
        y1, x1, y2, x2 = bb
        rec = Rectangle((x1, y1), x2-x1, y2-y1, facecolor='none', edgecolor='r')
        ax[0].add_patch(rec)
    
    ax[1].imshow(aug_img)
    for bb in aug_bboxes:
        y1, x1, y2, x2 = bb
        rec = Rectangle((x1, y1), x2-x1, y2-y1, facecolor='none', edgecolor='r')
        ax[1].add_patch(rec)
    plt.show()


def check_results(img, boxes, aug_type, classes=None):
    if aug_type == 'hflip':
        imcheck = Image.open('data/augmented/flipped.png')
        bbcheck = np.load('data/augmented/flipped.npy')
        assert np.array_equal(np.array(imcheck), np.array(img)), 'Horizontal flip is wrong!'
        assert np.array_equal(np.array(boxes), bbcheck), 'Horizontal flip is wrong!'
        print('Horizontal flip is working')

    elif aug_type == 'resize':
        imcheck = Image.open('data/augmented/resized.png')
        bbcheck = np.load('data/augmented/resized.npy')
        assert np.array_equal(np.array(imcheck), np.array(img)), 'Resizing is wrong!'
        assert np.array_equal(np.array(boxes), bbcheck), 'Resizing is wrong!'
        print('Resizing is working')

    elif aug_type == 'random_crop':
        imcheck = Image.open('data/augmented/cropped.png')
        bbcheck = np.load('data/augmented/cropped_bb.npy')
        clcheck = np.load('data/augmented/cropped_cl.npy')
        assert np.array_equal(np.array(imcheck), np.array(img)), 'Cropping is wrong!'
        assert np.array_equal(np.array(boxes), bbcheck), 'Cropping is wrong!'
        assert np.array_equal(np.array(classes), clcheck), 'Cropping is wrong!'
        print('Cropping is working')
    return